In [1]:
datadir = "../data/"
datafilename_network = "Link_info_network_and_proj.xlsx"
datafilename_proj = 'Certainty.xlsx'
datafilepath_network = datadir + datafilename_network
datafilepath_proj = datadir + datafilename_proj

from itertools import combinations
import pandas as pd
import numpy as np
import math
import torch
from torch.utils.data import Subset
import wandb


# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="CGN to predict AADT",

#     # track hyperparameters and run metadata
#     config={
#      "hidden_layer_sizes": [12,24,48,64],
#     }
# )


network_cols_used = ['LINK','ANODE','BNODE','LENGTH','A X_COORD','A Y_COORD','B X_COORD','B Y_COORD','LANES_AB','LEFT_AB', 'RIGHT_AB','SPEED_AB','FSPD_AB','CAP_AB','LANES_BA','LEFT_BA','RIGHT_BA','SPEED_BA','FSPD_BA','CAP_BA'
]

# all 30000 links in the general network
df_network= pd.read_excel(datafilepath_network,sheet_name='Link_Info',usecols=network_cols_used).dropna(subset=['LINK'])

#650 links in the 6 projects
df_proj= pd.read_excel(datafilepath_network, sheet_name='Project_Links')

import networkx as nx

# Create a directed graph for the entire network
G_network = nx.DiGraph()

Anodes = df_network['ANODE'].tolist
Bnodes = df_network['BNODE'].tolist

# Add edges_ab with attributes
edges = [
    (row['ANODE'], row['BNODE'], {
        'length': row['LENGTH'], 
        '#lanes': row['LANES_AB'], 
        'speed': row['SPEED_AB'], 
        'FSPD': row['FSPD_AB'],
        'capacity': row['CAP_AB'],
        'Link ID': row['LINK'],
        'AADT Before': 0,
        'auto volume before': 0,
        'VMT before': 0
    })
    for _, row in df_network.iterrows()
]

# Add edges_ba only if lanes_ba != 0
edges += [
    (row['BNODE'], row['ANODE'], {
        'length': row['LENGTH'], 
        '#lanes': row['LANES_BA'], 
        'speed': row['SPEED_BA'], 
        'FSPD': row['FSPD_BA'],
        'capacity': row['CAP_BA'],
        'Link ID': row['LINK'],
        'AADT Before': 0,
        'auto volume before': 0,
        'VMT before': 0
    })
    for _, row in df_network.iterrows() if row['LANES_BA'] != 0
]

# List of pairs of integers that corresond to eges in project
proj_links = list(zip(df_proj['ANODE'], df_proj['BNODE']))


# Identifying edges in G_network that correspond to the list of pairs
# proj_edges = [(u, v) for u, v in proj_links if G_network.has_edge(u, v)]

# for project links, the "AADT Before","auto voulme before" and "VMT before" attributes are known and not zero
for i, item in enumerate(edges):
    if (edges[i][0], edges[i][1]) in proj_links:
        idx = proj_links.index((edges[i][0], edges[i][1]))
        edges[i][2]['AADT Before'] = df_proj.loc[idx]['AADT(2010)-B']
        edges[i][2]['auto volume before'] = df_proj.loc[idx]['auto volume(2010)-B']
        edges[i][2]['VMT before'] = df_proj.loc[idx]['VMT-B']
        
    


G_network.add_edges_from(edges)


# Creating the subgraph with only project edges
G_proj = G_network.edge_subgraph(proj_links)

# Now we create a directed line graph of the original network graph G_network
# In the line graph G_network_dual, each vertex corresponds to a directed edge in G_network
# Let uv, vw be two vertices in G_network_dual. (uv, vw) is a directed edge in G_network_dual iff (u,v), (v,w) are 
# two directed edges in G_network
# This is equivalent to say that each directed edge in G_network_dual corresponds to a length =2 directed path in G_network

G_network_dual = nx.line_graph(G_network) # dual graph of G_network
G_proj_dual = nx.line_graph(G_proj) # dual graph of G_proj


node_attrs = {}
for edge in G_network_dual.nodes:
    u, v = edge
    edge_data = G_network[u][v]  # Get the attributes of the edge from the original graph
    node_attrs[edge] = f"length: {edge_data['length']}, #lanes: {edge_data['#lanes']}, speed: {edge_data['speed']}, FSPD: {edge_data['FSPD']}, capacity: {edge_data['capacity']}, Link ID: {edge_data['Link ID']}, AADT Before: {edge_data['AADT Before']}, auto volume before: {edge_data['auto volume before']}, VMT before: {edge_data['VMT before']}"

# Convert dictionary to DataFrame
def parse_attributes(attr_string):
    attributes = {}
    for item in attr_string.split(', '):
        key, value = item.split(': ')
        attributes[key] = float(value)
    return attributes

# Create a DataFrame
df_node_attr_network = pd.DataFrame(
    [dict(**parse_attributes(value), index=key) for key, value in node_attrs.items()],
    index=[key for key in node_attrs.keys()]
)

# the dataframe that contains all node attributes in the dual network graph
df_node_attr_network = df_node_attr_network.iloc[:, :-1]

df_network_link_id = df_node_attr_network['Link ID']

# reorder columns of the data frame so that Link ID is the last column
df_node_attr_network = df_node_attr_network.iloc[:,[0,1,2,3,4,6,7,8,5]]

# df_node_attr_network = df_node_attr_network.drop(['Link ID'], axis = 1)
# Mapping node labels in G_network_dual, which are pairs of integers that coorespond to node labels in G_network,
# to the natural numbers 
node_mapping = {node: i + 1 for i, node in enumerate(G_network_dual.nodes())}
# Step 3: Construct a new graph with edges represented by mapped values
mapped_G_network_dual = nx.DiGraph()

# Add nodes with mapped attributes
for original_node in G_network_dual.nodes(data=True):
    original_label, attributes = original_node
    new_label = node_mapping[original_label]
    
    # Add the new node with the same attributes
    mapped_G_network_dual.add_node(new_label, **attributes)


# Add edges to the mapped graph using the new labels
for u, v in G_network_dual.edges():
    new_u = node_mapping[u]
    new_v = node_mapping[v]
    mapped_G_network_dual.add_edge(new_u, new_v)

# Identifying vertices in G_network_dual that correspond to links in G_proj
proj_nodes = [pair for pair in proj_links if pair in G_network_dual.nodes()]

# Keep the first occurrence of each unique element in proj_nodes
proj_nodes_unique = list(dict.fromkeys(proj_nodes))


In [2]:

from torch_geometric.nn import GCNConv
import torch.nn as nn
import torch.nn.functional as F

node_attr_used = df_node_attr_network.columns[:-1]
# num_attr = len(node_attr_used)

num_attr = 6

num_node_label = 3

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(num_attr, hidden_channels)
        self.conv2 = GCNConv(hidden_channels,hidden_channels)
        self.conv3 = GCNConv(hidden_channels, num_attr)
        self.head = nn.Linear(num_attr, num_node_label)
        # add a multi-layer perceptron?
        # reference examples of using GNN for classification eg. citation network
        # can pretty much reuse all the previous layers (backbone) except for the last layer (layer head) which is specific to the application 

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()

        x = self.conv2(x, edge_index)
        x = x.relu()

        x = self.conv3(x, edge_index)
        x = x.relu()
        
        x = self.head(x)
        
        return x

class MLP(nn.Module):
    def __init__(self, input_size= 64, output_size = 32, hidden=256):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden = hidden
        self.net = nn.Sequential(
            nn.Linear(self.input_size, self.hidden),
            nn.ReLU(),
            nn.Linear(self.hidden, self.output_size)
        )
        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0.00, std=0.01)

    def forward(self, y):
        return self.net(y)



/home/duantu/soft/transportation_venv/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
# create a list of all sheet names

num_projs = 6
proj_list = list(range(1,num_projs+1))
def generate_combinations(proj_list):
    all_combinations = []
    # Loop through lengths from 1 to 6
    for length in range(1, 1+num_projs):
        # Generate combinations of the current length
        comb = combinations(proj_list, length)
        # Convert each combination to a string, add "p", and add to the list
        all_combinations.extend(['P' + ''.join(map(str, c)) for c in comb])
    return all_combinations
# Generate all combinations
sheet_names = generate_combinations(proj_list)

# Insert "P0" at the beginning of the list
# sheet_names.insert(0, "P0")


In [4]:
# import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

# Scale all node attributes to be in the range of (0,1)

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


# scale each column with its own scaler
scaled_node_attr_network = np.zeros(df_node_attr_network.iloc[:,:-1].shape)
for ii in range(df_node_attr_network.iloc[:,:-(1+3)].shape[1]):
    scaler = MinMaxScaler()
    scaled_node_attr_network[:,ii:ii+1] = scaler.fit_transform(df_node_attr_network.iloc[:,ii:ii+1])

# put the scaled data into a dataframe
df_scaled_node_attr_network = pd.DataFrame(scaled_node_attr_network,index=df_node_attr_network.index, columns=node_attr_used)

# define a scaler that works for array of any length
def scaler_minmax(array):
    array = np.asarray(array, dtype = float)
    # mean = np.mean(array, axis=0)
    min = np.min(array, axis=0)
    # std = np.std(array, axis=0)
    max_minus_min = np.max(array,axis=0)-np.min(array,axis=0)

    scaled_array = (array-min)/max_minus_min
    return scaled_array, min, max_minus_min

# scale AADT_before, auto volume before, and vmt before
aadt_before_min = scaler_minmax(np.asarray(df_node_attr_network[df_node_attr_network['AADT Before']!=0]["AADT Before"], dtype = float))[1]
aadt_before_max_minus_min = scaler_minmax(np.asarray(df_node_attr_network[df_node_attr_network['AADT Before']!=0]["AADT Before"], dtype = float))[2]
auto_vol_before_min = scaler_minmax(np.asarray(df_node_attr_network[df_node_attr_network['auto volume before']!=0]["auto volume before"], dtype = float))[1]
auto_vol_before_max_minus_min = scaler_minmax(np.asarray(df_node_attr_network[df_node_attr_network['auto volume before']!=0]["auto volume before"], dtype = float))[2]
vmt_before_min = scaler_minmax(np.asarray(df_node_attr_network[df_node_attr_network['VMT before']!=0]["VMT before"], dtype = float))[1]
vmt_before_max_minus_min = scaler_minmax(np.asarray(df_node_attr_network[df_node_attr_network['VMT before']!=0]["VMT before"], dtype = float))[2]

aadt_before_array_scaled = (df_node_attr_network[df_node_attr_network['AADT Before']!=0]["AADT Before"]-aadt_before_min)/aadt_before_max_minus_min
df_scaled_node_attr_network["AADT Before"].loc[aadt_before_array_scaled.index] = aadt_before_array_scaled

auto_vol_before_array_scaled = (df_node_attr_network[df_node_attr_network['auto volume before']!=0]["auto volume before"]-auto_vol_before_min)/auto_vol_before_max_minus_min
df_scaled_node_attr_network["auto volume before"].loc[auto_vol_before_array_scaled.index] = auto_vol_before_array_scaled

vmt_before_array_scaled = (df_node_attr_network[df_node_attr_network['VMT before']!=0]["VMT before"]-vmt_before_min)/vmt_before_max_minus_min
df_scaled_node_attr_network["VMT before"].loc[vmt_before_array_scaled.index] = vmt_before_array_scaled


# TO unscale after training, use scaler.inverse_transform

df_scaled_node_attr_proj = df_scaled_node_attr_network.loc[list(G_proj_dual)]







# Prepare the dataset 

proj_cols_used = ['Link ID','ANODE','BNODE','A X_COORD','A Y_COORD','B X_COORD','B Y_COORD','Link Length(miles)','# of lanes-A','Capacity-A (veh/h)',
            'auto volume(2010)-A','AADT(2010)-B','AADT(2010)-A','Speed(mph)-A','VMT-A']
proj_features_used = ['']
dataset = []


# x = node features
# The entire network has node features
x = torch.tensor(np.array(df_scaled_node_attr_network), dtype=torch.float)


# edge index is shared among all samples
# Get the list of edges as tuples (source, target)
# the entire network has edge_index 
edges = list(mapped_G_network_dual.edges())

# Separate source and target nodes
source_nodes = [edge[0] for edge in edges]
target_nodes = [edge[1] for edge in edges]

# Create the edge_index tensor: shape [2, num_edges]
edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)-1


for ii in range(len(sheet_names)):
    # read the correct sheet 
    df_sample= pd.read_excel(datafilepath_proj,sheet_name=sheet_names[ii],usecols=proj_cols_used).dropna(subset=['Link ID'])
    df_sample = df_sample.astype('float64')

    # create a df to store scaled labels
    df_sample_label_scaled = pd.DataFrame(-1, index=df_sample.index, columns=['scaled AADT after','scaled auto volume after','scaled VMT after'])

    # scale AADT after, auto vol after and vmt after
    df_sample_label_scaled['scaled AADT after'] = (df_sample['AADT(2010)-A']-aadt_before_min)/aadt_before_max_minus_min
    df_sample_label_scaled['scaled auto volume after'] = (df_sample['auto volume(2010)-A']-auto_vol_before_min)/auto_vol_before_max_minus_min
    df_sample_label_scaled['scaled VMT after'] = (df_sample['VMT-A']-vmt_before_min)/vmt_before_max_minus_min
    
    


################################################################################
    # node attributes are defined by x above 
################################################################################   
    # # edge index
    # already defined as above
###################################################################################
    
    # Create the label tensor y
        
    # -1 for unlabeled nodes, correct class labels (positive real value) for labeled nodes in G_proj_dual
    # create a all -1 dataframe to store labels of the entire network
    df_scaled_labels_network = pd.DataFrame(-1, index = df_scaled_node_attr_network.index, columns = ['Link ID','scaled AADT after','scaled auto volume after','scaled VMT after'])
    network_links = list(G_network_dual.nodes)

    df_scaled_labels_network['Link ID'] = df_network_link_id

    for index in range(1, len(df_sample)+1):  # Loop through each row in df_sample
        link_id = df_sample.loc[index, 'Link ID']  # Get 'Link ID' for the current row
    
    # Find the row in df_scaled_labels_network where 'Link ID' matches
        row_idx = df_scaled_labels_network[df_scaled_labels_network['Link ID'] == link_id].index
    
        if not row_idx.empty:  # If a match is found
        # Assign the value of 'AADT(2010)-A' from df_sample to 'AADT_after' in df_scaled_labels_network
            df_scaled_labels_network = df_scaled_labels_network.astype('float64')
            df_scaled_labels_network.loc[row_idx, 'scaled AADT after'] = df_sample_label_scaled.loc[index, 'scaled AADT after']
            df_scaled_labels_network.loc[row_idx, 'scaled auto volume after'] = df_sample_label_scaled.loc[index, 'scaled auto volume after']
            df_scaled_labels_network.loc[row_idx, 'scaled VMT after'] = df_sample_label_scaled.loc[index, 'scaled VMT after']



    
    y = torch.tensor(np.array(df_scaled_labels_network[['scaled AADT after','scaled auto volume after','scaled VMT after']]), dtype=torch.float)  

##################################################################################  
    # Create the train_mask so that training only happens on project links
    train_mask = pd.DataFrame(False, index = df_scaled_node_attr_network.index, columns = ['Masked'])

    row_idx = df_scaled_labels_network[df_scaled_labels_network['scaled AADT after'] != -1].index

    train_mask.loc[row_idx] = True
    train_mask = torch.tensor(train_mask.values, dtype=torch.bool)

    new_data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask)


    # Append all 64 samples
    dataset.append(new_data)


/tmp/ipykernel_959023/622721769.py:40: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_scaled_node_attr_network["AADT Before"].loc[aadt_before_array_scaled.index] = aadt_before_array_scaled
/tmp/ipykernel_959023/622721769.py:43: FutureWarni

In [5]:
# define our data loader
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    
    def __init__(self,data):
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]['x']
        edge_index = self.data[index]['edge_index']
        target = self.data[index]['y']
        return x, edge_index, target
    

In [6]:

val_indices =[59, 18, 27, 13, 32, 57, 54, 47, 61, 17, 43, 28, 62, 44, 14, 26, 6, 41]

val_dataset = Subset(dataset, val_indices)

val_dataset = CustomDataset(val_dataset)

val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [7]:
# Training settings
# model = GCN(hidden_channels=24)
# state = torch.load( "./checkpoint.pth.tar")
# model.load_state_dict(state["model_state"])
# criterion = torch.nn.MSELoss()



model = GCN(hidden_channels=24)
state = torch.load( "./savedModels/GCN model epoch_198.pth")
model.load_state_dict(torch.load('./savedModels/GCN model epoch_198.pth', weights_only=True))
criterion = torch.nn.MSELoss()


In [8]:
output_indices = list(np.array(df_node_attr_network.index)[train_mask.squeeze()])


proj_order_list = list(zip(df_sample[:]['ANODE'],df_sample[:]['BNODE']))
# print(proj_order_list)


#### I THINK THIS SORTING OF output_indices WORKS!!!!
sorted_output_indices = [item for item in proj_order_list if item in output_indices]

### NEXT: put predicted AADT and real AADT together 


# NOTICE: 27 output links are not in proj links 

In [9]:
sum_target_2_norm_squared_AADT = 0
for i, (x, edge_index, target) in enumerate(val_loader):
    sum_target_2_norm_squared_AADT += criterion(target[0][train_mask.squeeze()][:,0], torch.zeros(target[0][train_mask.squeeze()][:,0].shape))
normalizing_const_ave_val_AADT = sum_target_2_norm_squared_AADT/len(val_loader)


total_val_loss_AADT = 0
df_rel_err = pd.DataFrame(columns = val_indices, index = np.array(df_node_attr_network['Link ID'],dtype = int).reshape(-1,1)[np.array(train_mask)==1])

df_AADT_comparison = pd.DataFrame(columns = [sheet_names[i] for i in df_rel_err.columns],index = proj_order_list)

for i, (x, edge_index, target) in enumerate(val_loader):
    
    training_attr_tensor = torch.cat((x[0][:,:2],torch.unsqueeze(x[0][:,4],1), x[0][:,-3:]),1)
    out = model(training_attr_tensor, edge_index[0]).squeeze() 
    normalizing_constant_single_val_set_AADT = criterion(target[0][train_mask.squeeze()][:,0], torch.zeros(target[0][train_mask.squeeze()][:,0].shape))
    
    loss_AADT = criterion(out[train_mask.squeeze()][:,0], target[0][train_mask.squeeze()][:,0])
    # print(out[train_mask.squeeze()])
    # print(target[0][train_mask.squeeze()][:,0])
    relative_loss_AADT = loss_AADT/normalizing_constant_single_val_set_AADT.item()
    
    print(f'\nVal sample # {val_indices[i]} Absolute MSE (AADT): {format(loss_AADT,".2e")}') 
    print(f'Val sample # {val_indices[i]} Relative MSE (AADT): {format(relative_loss_AADT,".2e")}')

    #########
    unscaled_output = (out[train_mask.squeeze()].detach().numpy()[:,0]*aadt_before_max_minus_min)+aadt_before_min
    unscaled_node_label = (target[0][train_mask.squeeze()].detach().numpy()[:,0] *aadt_before_max_minus_min)+aadt_before_min

    # I would like to create a table w/ 3 columns here: "unscaled_output", "unscaled_node_label", "AADT(2010)-A" from the project data file
    # unscaled_output and unscaled_node_label needs to be resorted to match the ordre of "AADT(2010)-A"
    # Supposedly, sorting according to the "sorted_output_indices" variable would work, but it's not working 
    # THIS IS WHERE I'M HAVING TROUBLE 
    df_unscaled_output = pd.DataFrame(unscaled_output,index = output_indices)
    df_unscaled_output_sorted = df_unscaled_output.loc[sorted_output_indices]

    df_unscaled_node_label = pd.DataFrame(unscaled_node_label,index = output_indices)
    df_unscaled_node_label_sorted = df_unscaled_node_label.loc[sorted_output_indices]

    
    ##### pull real AADT data 


    
    df_rel_err.iloc[:,i] = abs(unscaled_output-unscaled_node_label)/unscaled_node_label

    num = 0
    for j in range(len(unscaled_node_label)):
        if abs(unscaled_output[j]-unscaled_node_label[j])/unscaled_node_label[j]>0.9:
            num+=1
    # print([abs(unscaled_output-unscaled_node_label)/unscaled_node_label][0][491])
    print('validation sample #', val_indices[i], 'has ',num, 'links with relative error > 0.9')
    ########
    
    total_val_loss_AADT += loss_AADT.item()
      
# normalizing constant: squared l2 norm divided by length of y[train_mask] 
relative_ave_val_mse_AADT = total_val_loss_AADT/len(val_loader)/normalizing_const_ave_val_AADT.item()
print(f"\nRelative average validation set mse (AADT) = {total_val_loss_AADT/len(val_loader):.2e} / {normalizing_const_ave_val_AADT.item():.2e} = {relative_ave_val_mse_AADT:.2e}")

# make a copy of the data frame that has all the relative error info of each validation sample 
import copy
df_rel_err_copy = copy.copy(df_rel_err)


Val sample # 59 Absolute MSE (AADT): 4.16e-01
Val sample # 59 Relative MSE (AADT): 3.59e-01
validation sample # 59 has  66 links with relative error > 0.9

Val sample # 18 Absolute MSE (AADT): 1.67e-01
Val sample # 18 Relative MSE (AADT): 3.59e-01
validation sample # 18 has  168 links with relative error > 0.9

Val sample # 27 Absolute MSE (AADT): 1.38e-01
Val sample # 27 Relative MSE (AADT): 3.37e-01
validation sample # 27 has  278 links with relative error > 0.9

Val sample # 13 Absolute MSE (AADT): 1.42e-01
Val sample # 13 Relative MSE (AADT): 4.31e-01
validation sample # 13 has  285 links with relative error > 0.9

Val sample # 32 Absolute MSE (AADT): 1.47e-01
Val sample # 32 Relative MSE (AADT): 4.04e-01
validation sample # 32 has  267 links with relative error > 0.9

Val sample # 57 Absolute MSE (AADT): 4.07e-01
Val sample # 57 Relative MSE (AADT): 3.80e-01
validation sample # 57 has  146 links with relative error > 0.9

Val sample # 54 Absolute MSE (AADT): 2.69e-01
Val sample #

In [10]:
# # pd.DataFrame(columns = [sheet_names[i] for i in df_rel_err.columns],index = df_rel_err.index)
df_unscaled_output = pd.DataFrame(unscaled_output,index = output_indices)
# # abs(unscaled_node_label-unscaled_output)/unscaled_node_label
# # sorted_output_indices
df_unscaled_output.loc[sorted_output_indices]
# df_unscaled_output


,0
"(8921.0, 8981.0)",69284.460938
"(9137.0, 8980.0)",67931.203125
"(9027.0, 9141.0)",65643.867188
"(9141.0, 9267.0)",62730.582031
"(9264.0, 9137.0)",69903.500000
...,...
"(12061.0, 42650.0)",83747.203125
"(42650.0, 42659.0)",75259.734375
"(42658.0, 42660.0)",65924.656250
"(42660.0, 12182.0)",65563.921875


In [11]:
# make a copy of the relative error dataframe 


# Compute the sum of values > 0.9 for each column
num_large_error = df_rel_err.apply(lambda col: (col > 0.9).sum()).sort_values(ascending=False).index
df_rel_err_sorted_by_set = df_rel_err[num_large_error]


df_rel_err_sorted_by_set.columns =  [sheet_names[i] for i in df_rel_err_sorted_by_set.columns] 


val_sample_num_large_err_links = pd.DataFrame(columns=["#links with > 0.9 relative err"],index = df_rel_err_sorted_by_set.columns)
for i in range(len(val_indices)):

    val_sample_num_large_err_links.iloc[i] = sum(df_rel_err_sorted_by_set.iloc[:,i]>0.9)

# print the that shows # of large err links of each validation sample 
print(val_sample_num_large_err_links)


for i in range(len(val_indices)):

    df_rel_err_sorted_by_set.iloc[:,i] = list(zip(df_rel_err_sorted_by_set.iloc[:,i].sort_values(ascending=False).index, df_rel_err_sorted_by_set.iloc[:,i].sort_values(ascending=False))) 
df_rel_err_sorted_by_set.index = range(len(df_rel_err_sorted_by_set))

# print the dataframe that contains all relative err info of all val sets 
print(df_rel_err_sorted_by_set)



        #links with > 0.9 relative err
P12                                346
P26                                312
P36                                295
P25                                285
P136                               278
P1236                              275
P235                               267
P135                               251
P1234                              203
P45                                168
P12346                             146
P145                               141
P1245                              137
P1345                              123
P2456                              105
P23456                              77
P12456                              66
P123456                             47
                               P12                             P26  \
0         (81748, 2610.3154296875)      (81748, 2609.509033203125)   
1      (81750, 492.61322021484375)     (81750, 492.46075439453125)   
2      (22655, 103.04730224609375)     (81778, 19

In [12]:
# count how many times link #81779 ended up having relative err >0.9

# Flatten the DataFrame to extract IDs and values
flattened = pd.DataFrame(
    [(id, val) for col in df_rel_err_sorted_by_set.columns for id, val in df_rel_err_sorted_by_set[col]],
    columns=['Link ID', 'relative error']
)

# df_large_err_links

# Filter rows where Value > 0.9
filtered = flattened[flattened['relative error'] > 0.9]


# Count occurrences of each ID
link_id_counts = filtered['Link ID'].value_counts()

print(link_id_counts)

Link ID
2215     36
2333     36
2223     36
2218     36
2216     36
         ..
81838     7
10907     6
81061     4
10813     4
81159     2
Name: count, Length: 351, dtype: int64


In [97]:
large_err_links = flattened[flattened['relative error']>5].drop_duplicates(subset=['Link ID'])['Link ID']
large_err_links = df_network_link_id[df_network_link_id.isin(large_err_links)].index

In [99]:

pd.set_option('display.max_colwidth', None)
print(large_err_links)
# with open("output.txt", "w") as f:
#     f.write(str(list(large_err_links)))

Index([  (6256.0, 6718.0),   (6466.0, 6543.0),   (6466.0, 6461.0),
         (6461.0, 6466.0),   (6547.0, 6543.0),   (6543.0, 6547.0),
         (6543.0, 6466.0),   (6719.0, 6253.0),  (8220.0, 80767.0),
         (9884.0, 9912.0),   (8854.0, 8921.0),   (8913.0, 8853.0),
         (9912.0, 9938.0),  (9561.0, 81189.0),  (81189.0, 9601.0),
       (81185.0, 81187.0), (81187.0, 81186.0),  (81188.0, 9750.0),
        (81186.0, 9772.0), (10020.0, 10165.0),   (9974.0, 9885.0),
         (9938.0, 9975.0), (10005.0, 10037.0), (81138.0, 81139.0),
       (81134.0, 81135.0), (10038.0, 10007.0), (81132.0, 81133.0),
        (10184.0, 9997.0), (10235.0, 10252.0), (10252.0, 10261.0),
       (81152.0, 11387.0), (81156.0, 81157.0), (11436.0, 11409.0),
       (42652.0, 12050.0), (12061.0, 42650.0), (42650.0, 42659.0),
       (42658.0, 42659.0), (42659.0, 12126.0), (42662.0, 42661.0),
       (81126.0, 81125.0), (81120.0, 81121.0), (81122.0, 13820.0),
       (81124.0, 81123.0), (81266.0, 81137.0)],
      dtype='o

In [100]:
# df_large_err_links contains all links (among the 650 project links) that had relative error >0.9 at least once among the 18 validation sets
# this cell prints their information 
# links in this table is ordered from the most number of times it had > 0.9 relative error to the least 
df_proj_all_info = pd.read_excel(datafilepath_proj,sheet_name=sheet_names[0])[1:]

df_large_err_links = df_proj_all_info[df_proj_all_info['Link ID'].astype(int).isin(link_id_counts.index)].drop_duplicates(subset='Link ID')


df_large_err_links = df_large_err_links.set_index('Link ID')  # Set 'Link ID' as the index
df_large_err_links.index = df_large_err_links.index.astype(int)
df_large_err_links = df_large_err_links.reindex(np.array(link_id_counts.index))
df_large_err_links.style


,Category,No.,Project,Link,ANODE,BNODE,A X_COORD,A Y_COORD,B X_COORD,B Y_COORD,Link Type,Dir,Link Length(miles),# of lanes-B,# of lanes-A,Capacity-B (veh/h),Capacity-A (veh/h),auto volume(2010)-B,AADT(2010)-B,auto volume(2010)-A,AADT(2010)-A,Speed(mph)-B,Speed(mph)-A,VMT-B,VMT-A,Project-Related Agency Cost-B,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Project-Related User Cost_B,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Agency Benefits,Unnamed: 45,Unnamed: 46,User Benefits,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Total Benefits (Agency+User),Unnamed: 63,Unnamed: 64
Link ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2215,Project,145.000000,5.000000,I-47,6461.000000,6462.000000,380341.100000,4596362.000000,380358.100000,4596871.000000,F,S,0.596516,1.000000,1.000000,870.000000,870.000000,6560.815781,8960.850652,7175.527017,9800.431545,51.440214,48.426902,1951032.263621,2133832.923283,3450131.953658,395822.822989,134937.684586,10501.780583,351.758670,3586382.579669,4529914.049431,238145.132377,181196.561977,511137.897527,272868.757626,327184.367424,98794.527776,1209985.550353,96279550.262598,121609470.746314,4864378.829853,4864378.829853,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2333,Project,142.000000,5.000000,I-47,6543.000000,6547.000000,381094.500000,4592347.500000,381099.200000,4590727.500000,F,S,2.019953,1.000000,1.000000,990.000000,990.000000,5824.946659,8152.253200,6370.711130,8916.073097,53.658509,52.207337,6010517.833960,6573669.272019,11683009.328076,1340355.034346,456932.734428,35561.654499,1191.142796,12144388.010406,15339421.449885,806418.954513,613576.857995,1542519.462486,815496.064303,432693.605360,304354.921325,3095064.053475,246276804.723390,311069087.857375,12442763.514295,12442763.514295,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2223,Project,143.000000,5.000000,I-47,6466.000000,6543.000000,380342.600000,4593135.000000,381094.500000,4592347.500000,F,S,1.360057,1.000000,1.000000,990.000000,990.000000,5765.675912,8092.982453,6305.887050,8851.249017,53.761036,52.345257,4017525.647130,4393945.018659,7866300.854341,902476.036415,307657.920856,23944.059729,802.009768,8176952.281646,10328204.032702,542970.901820,413128.161308,1030051.744053,544811.902837,387676.574897,203435.666615,2165975.888402,172348491.561825,217691179.337603,8707647.173504,8707647.173504,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2218,Project,146.000000,5.000000,I-47,6462.000000,6463.000000,380358.100000,4596871.000000,380378.100000,4597975.500000,F,S,1.384912,1.000000,1.000000,1050.000000,1050.000000,6542.578629,8942.613499,7155.581146,9780.485674,53.220443,51.594527,4520427.820311,4943966.273731,8010056.352410,918968.654039,313280.324380,24381.633920,816.666379,8326384.889133,10516950.451428,552893.615669,420678.018057,1164880.544609,611299.068511,404185.599872,228901.151550,2409266.364543,191707315.821458,242143063.107976,9685722.524319,9685722.524319,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2216,Project,144.000000,5.000000,I-47,6461.000000,6466.000000,380341.100000,4596362.000000,380342.600000,4593135.000000,F,S,4.020023,1.000000,1.000000,790.000000,790.000000,5369.929692,7442.687081,5873.061654,8140.024656,52.689170,50.394755,10920717.407981,11943926.702559,23251014.257697,2667515.974593,909367.546037,70773.249644,2370.560304,24169229.934823,30527845.764788,1604899.737947,1221113.830592,2828171.336087,1501912.411842,506850.307083,552992.966553,5389927.021565,428880947.737766,541714049.611928,21668561.984477,21668561.984477,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2219,Project,147.000000,5.000000,I-47,6463.000000,6464.000000,380378.100000,4597975.500000,380387.300000,4598784.000000,F,S,1.004881,1.000000,1.000000,1050.000000,1050.000000,6511.575468,9129.795327,7121.673166,9985.205378,53.183160,

In [ ]:

# sorted_output_indices = sorted(output_indices, key=lambda x: proj_order_list.index(x) if x in proj_order_list else float('inf'))
# print(sorted_output_indices)

# for i, item in enumerate(sorted_output_indices):
#     if item not in proj_order_list:
#         print(i)

# sorted_output_indices

# (11846.0, 11833.0) in proj_order_list
# (6552.0, 6549.0) in list(output_indices)

# # import torch
# from torch_geometric.data import Data
# from torch_geometric.loader import DataLoader

# # Scale all node attributes to be in the range of (0,1)

# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler

# # scaler = MinMaxScaler()
# scaler = StandardScaler()
# scaled_node_attr_network = scaler.fit_transform(df_node_attr_network.iloc[:,:-1])
# # TO unscale after training, use scaler.inverse_transform

# # put the scaled data into a dataframe
# df_scaled_node_attr_network = pd.DataFrame(scaled_node_attr_network,index=df_node_attr_network.index, columns=node_attr_used)
# df_scaled_node_attr_proj = df_scaled_node_attr_network.loc[list(G_proj_dual)]






# # # aa = scaler.inverse_transform(df_sample_scaled)
# # # list(zip(df_sample['AADT(2010)-A'],aa))
# real_aadt = pd.read_excel(datafilepath_proj,sheet_name=sheet_names[val_indices[0]],usecols=proj_cols_used).dropna(subset=['Link ID'])['AADT(2010)-A']

# unscaled_node_label = scaler.inverse_transform(target[0][train_mask.squeeze()].detach().numpy())[:,0]


# df_unscaled_node_label = pd.DataFrame(data = unscaled_node_label)
# df_unscaled_node_label.columns = ['unscaled_node_label']
# df_unscaled_node_label.index = df_node_attr_network.index[train_mask.squeeze()]
# sorted_df_unscaled_node_label = df_unscaled_node_label.reindex(sorted_output_indices)

# list(zip(sorted_df_unscaled_node_label, real_aadt))
# # # real_aadt

# unscaled = scaler.inverse_transform(df_scaled_node_attr_network)[:,5]
# unscaled = unscaled[train_mask.squeeze()]



# # convert unscaled_label into a data frame
# df_unscaled_node_label = pd.DataFrame(data = unscaled_node_label)
# df_unscaled_node_label.columns = ['unscaled_node_label']
# df_unscaled_node_label.index = df_node_attr_network.index[train_mask.squeeze()]
# sorted_df_unscaled_node_label = df_unscaled_node_label.reindex(sorted_output_indices)
# # df_real_label = pd.read_excel(datafilepath_proj,sheet_name=sheet_names[13],usecols=proj_cols_used).dropna(subset=['Link ID'])['AADT(2010)-A']
# # list(zip(sorted_df_unscaled_node_label['unscaled_node_label'],df_real_label))
# # # df_real_output['AADT(2010)-A']
# # # df_real_output

In [ ]:
# Check that we have correctly created the line graph

# print some node labels 
# for node, label in node_labels.items():
#     print(f"Node {node}: {label}")

# # Print a couple of edges and their attributes from the original graph
# print("Sample edges from the original graph with attributes:")
# for u, v, data in list(G_network.edges(data=True))[:5]:  # Print the first 5 edges as an example
#     print(f"Edge ({u}, {v}): {data}")

# # Print a few nodes and their attributes from the line graph
# print("Sample nodes in the line graph with their labels:")
# for node in list(G_network_dual.nodes)[:5]:  # Print the first 5 nodes as an example
#     print(f"Node {node}: {node_labels[node]}")


In [ ]:


# import networkx as nx
# 
# # Create a directed graph
# G = nx.DiGraph()

# # Add nodes
# G.add_nodes_from([1, 2, 3, 4])

# # Add edges with attributes
# G.add_edge(1, 2, weight=5)
# G.add_edge(2, 3, weight=3)
# G.add_edge(3, 4, weight=7)
# G.add_edge(4, 1, weight=2)
# G.add_edge(2, 4, weight=4)

# # # Display the edges with attributes
# # for u, v, data in G.edges(data=True):
# #     print(f"Edge from {u} to {v} with attribute: {data}")



# H = nx.line_graph(G)

# node_labels = {}
# for edge in H.nodes:
#     u, v = edge
#     node_labels[edge] = G[u][v]['weight']  # Get the weight from the original graph




# # Optionally, draw the graph
# import matplotlib.pyplot as plt

# pos = nx.spring_layout(G)
# nx.draw(G, pos, with_labels=True, node_color='lightblue', arrows=True)
# edge_labels = nx.get_edge_attributes(G, 'weight')
# nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)


# plt.show()

# posH = nx.spring_layout(H)
# nx.draw(H, posH, with_labels=True, node_color='lightgreen',arrows=True)

# # Draw the node labels (weights from the original graph)
# nx.draw_networkx_labels(H, posH, labels=node_labels, font_color='red')



# plt.show()





In [ ]:
# import itertools

# # find the 650 links in the network based on link ID
# proj_link_index = df_proj['Link ID'].apply(lambda x: df_network.index[df_network['LINK'] == x].to_list()).to_list()

# proj_link_index=list(itertools.chain.from_iterable(proj_link_index))

# # check if Anodes are the same
# Anode_same = np.all(df_network['ANODE'].iloc[proj_link_index].to_numpy() - df_proj['ANODE'].to_numpy()==0)

# # check if Bnodes are the same
# Bnode_same = np.all(df_network['BNODE'].iloc[proj_link_index].to_numpy() - df_proj['BNODE'].to_numpy()==0)

# # check if length are the same
# length_same = np.all(df_network['LENGTH'].iloc[proj_link_index].to_numpy() - df_proj['Link Length(meter)'].to_numpy()==0)

# A_X_Cord_same = np.all(df_network['A X_COORD'].iloc[proj_link_index].to_numpy() - df_proj['A X_COORD'].to_numpy()==0)
# A_Y_Cord_same = np.all(df_network['A Y_COORD'].iloc[proj_link_index].to_numpy() - df_proj['A Y_COORD'].to_numpy()==0)
# B_X_Cord_same = np.all(df_network['B X_COORD'].iloc[proj_link_index].to_numpy() - df_proj['B X_COORD'].to_numpy()==0)
# B_Y_Cord_same = np.all(df_network['B Y_COORD'].iloc[proj_link_index].to_numpy() - df_proj['B Y_COORD'].to_numpy()==0)


# # verify that all the A_node, B_note, length info about the project links are consistant across the two data files 
# print("Is it true that A_node, B_note, length info about the project links are consistant across the two data files? \n", Anode_same, Bnode_same, length_same)
# print(A_X_Cord_same, A_Y_Cord_same, B_X_Cord_same, B_Y_Cord_same)